In [1]:
%matplotlib inline
import os
import re
import datetime as dt
import numpy as np
import pandas as pd

pd.set_option('precision', 3)

import matplotlib.pyplot as plt
from args_tools_jupyter import args, createfolder, print_dict
from weather_wrangler import read_data, localP_2_seaP, localT_2_seaT
from scipy.interpolate import Rbf

# os.environ['PROJ_LIB'] = os.path.expanduser('~/anaconda3/share/proj')
from mpl_toolkits.basemap import Basemap
import idw
# print_dict(args)

In [2]:
ty_list = pd.read_csv(args.ty_list)
ty_list['Time of canceling'] = pd.to_datetime(ty_list['Time of canceling'])
ty_list['Time of issuing'] = pd.to_datetime(ty_list['Time of issuing'])

sta_list = pd.read_csv(args.sta_list)
sta_list.set_index('NO', inplace=True)
sta_list.index = sta_list.index.astype(str)
print(ty_list.head())
print(sta_list.head())

    En name Ch name     Time of issuing   Time of canceling
0     SAOLA      蘇拉 2012-07-31 21:00:00 2012-08-03 14:00:00
1    SOULIK      蘇力 2013-07-12 12:00:00 2013-07-13 23:00:00
2     TRAMI      潭美 2013-08-20 20:00:00 2013-08-22 08:00:00
3  KONG-REY      康芮 2013-08-28 11:00:00 2013-08-29 17:00:00
4     USAGI      天兔 2013-09-20 08:00:00 2013-09-22 08:00:00
        Unnamed: 0 Station name  Height(m)  Longitude  Latitude City  \
NO                                                                     
466880           0           板橋        9.7    121.442    24.998  新北市   
466900           1           淡水       19.0    121.449    25.165  新北市   
466910           2           鞍部      825.8    121.530    25.183  臺北市   
466920           3           臺北        6.3    121.515    25.038  臺北市   
466930           4          竹子湖      607.1    121.544    25.162  臺北市   

       origin date  
NO                  
466880  1972/03/01  
466900  1942/01/01  
466910  1937/01/01  
466920  1896/01/01  
466930  1

##### PS01 測站氣壓(hPa)
##### TX01 氣溫(℃)
##### RH01 相對濕度(%)
##### WD01 平均風風速(m/s)
##### WD02 平均風風向(360 degree)
##### PP01 降水量(mm)

In [ ]:
timesteps = ty_list['Time of canceling'] - ty_list['Time of issuing']

# x y meshgrid for interpolation
X, Y = np.meshgrid(np.linspace(args.O_x[0], args.O_x[1], args.O_shape[0]), np.linspace(args.O_y[0], args.O_y[1], args.O_shape[1]))
XY = np.stack([Y.flatten(), X.flatten()], axis=1)

column = pd.Index(np.linspace(args.O_x[0], args.O_x[1], args.O_shape[0]), name='lon')
index = pd.Index(np.linspace(args.O_y[0], args.O_y[1], args.O_shape[1]), name='lat')

# plot setting
plot_fig = 1
m = Basemap(projection='cyl', resolution='h', llcrnrlat=args.O_y[0], urcrnrlat=args.O_y[1], 
            llcrnrlon=args.O_x[0], urcrnrlon=args.O_x[1])

args.weather_figures_folder = os.path.join(args.weather_folder, '03_figures', 'Taiwan')
createfolder(args.weather_figures_folder)

fig = plt.figure(figsize=(18,12),dpi=args.figure_dpi)

for idx, timestep in enumerate(timesteps):
    print(ty_list.iloc[idx,1])
    timelines = [ty_list['Time of issuing'][idx]+dt.timedelta(hours=x) for x in range(timestep.days*24+timestep.seconds//3600+1)]
    data_man = os.path.join(args.weather_raw_data_folder, str(ty_list.iloc[idx,2].year)+'.Data', 
                            str(ty_list.iloc[idx,2].year)+'_Station_hr_man', ty_list.iloc[idx,0]+'.txt')
    data_auto = os.path.join(args.weather_raw_data_folder, str(ty_list.iloc[idx,2].year)+'.Data', 
                             str(ty_list.iloc[idx,2].year)+'_Station_hr_auto', ty_list.iloc[idx,0]+'.txt')
    data = read_data(data_man, data_auto, sta_list)

    for j in range(len(timelines)-1):
        sta_x1 = data.loc[timelines[j]].lon.to_numpy()
        sta_y1 = data.loc[timelines[j]].lat.to_numpy()
        sta_x2 = data.loc[timelines[j+1]].lon.to_numpy()
        sta_y2 = data.loc[timelines[j+1]].lat.to_numpy()
        
        for p, k in enumerate(data.columns[1:-3]):
            output_folder = os.path.join(args.weather_wrangled_data_folder, k)
            if not os.path.exists(output_folder):
                print(output_folder)
                createfolder(output_folder)

            sta_xy1 = np.stack([sta_y1, sta_x1], axis=1)
            sta_xy2 = np.stack([sta_y2, sta_x2], axis=1)
            
            if k == 'PS01':
                z1 = localP_2_seaP(p=data.loc[timelines[j], k].to_numpy(), h=data.loc[timelines[j], 'height'].to_numpy(), 
                                   t=data.loc[timelines[j], 'TX01'].to_numpy())
                z2 = localP_2_seaP(p=data.loc[timelines[j+1], k].to_numpy(), h=data.loc[timelines[j+1], 'height'].to_numpy(), 
                                   t=data.loc[timelines[j+1], 'TX01'].to_numpy())
#             elif k == 'TX01':
#                 z1 = localT_2_seaT(h=data.loc[timelines[j], 'height'].to_numpy(), t=data.loc[timelines[j], 'TX01'].to_numpy())
#                 z2 = localT_2_seaT(h=data.loc[timelines[j+1], 'height'].to_numpy(), t=data.loc[timelines[j+1], 'TX01'].to_numpy())
            else:
                z1 = data.loc[timelines[j], k].to_numpy()
                z2 = data.loc[timelines[j+1], k].to_numpy()
            
            # Z1
            idw_tree = idw.tree(sta_xy1, z1)
            Z1 = idw_tree(XY).reshape(args.O_shape[1], args.O_shape[0])
            Z1[Z1 < 0] = 0
            
            if k == 'RH01':
                Z1[Z1 > 100] = 100
            Z1 = pd.DataFrame(Z1, columns=column, index=index)
            
            # Z2
            idw_tree = idw.tree(sta_xy2, z2)
            Z2 = idw_tree(XY).reshape(args.O_shape[1], args.O_shape[0])
            Z2[Z2 < 0] = 0
            
            if k == 'RH01':
                Z2[Z2 > 100] = 100
            Z2 = pd.DataFrame(Z2, columns=column, index=index)
            
            if plot_fig:
                ax = fig.add_subplot(2, int(len(data.columns[1:-3])/2), p+1)
                if k == 'WD02':
                    idw_tree = idw.tree(sta_xy1, z1)
                    Xw, Yw = np.meshgrid(np.linspace(args.O_x[0], args.O_x[1], 28), np.linspace(args.O_y[0], args.O_y[1], 22))
                    XYw = np.stack([Yw.flatten(), Xw.flatten()], axis=1)
                    Z3 = idw_tree(XYw).reshape(28, 22)
                    Z3[Z3 < 0] = 0
                    
                    z4 = data.loc[timelines[j], 'WD01'].to_numpy()
                    idw_tree = idw.tree(sta_xy1, z4)
                    Z4 = idw_tree(XYw).reshape(28, 22)
                    Z4[Z4 < 0] = 0
                    
                    U = np.sin(Z3)*Z4
                    V = np.cos(Z3)*Z4
                    _ = m.readshapefile(args.TW_map_file, name='Taiwan', linewidth=0.25, drawbounds=True, color='k', ax=ax)
                    Q = ax.quiver(Xw, Yw, U, V, units='width', color='b')
                    qk = ax.quiverkey(Q, 0.9, 0.9, 2, r'$2 \frac{m}{s}$', labelpos='E', coordinates='figure')
                else:
                    _ = m.readshapefile(args.TW_map_file, name='Taiwan', linewidth=0.25, drawbounds=True, color='k', ax=ax)
                    cs = m.contourf(x=X, y=Y, data=Z1, ax=ax, levels=args[k+'_level'], colors=args[k+'_cmap'])
                    cbar = fig.colorbar(cs, ax=ax)
                    cbar.ax.tick_params(labelsize=10)
                    
                ax.set_title(args.weather_names[p])
                ax.tick_params('both', labelsize=10)
                ax.set_xlabel(r'longtitude($^o$)',fontdict={'fontsize':10})
                ax.set_ylabel(r'latitude($^o$)',fontdict={'fontsize':10})
                _ = ax.set_xticks(ticks = np.linspace(args.O_x[0], args.O_x[1], 5))
                _ = ax.set_yticks(ticks = np.linspace(args.O_y[0], args.O_y[1], 5))
            
            for i in range(7):
                Z = Z1+(Z2-Z1)*i/5
                output_path = os.path.join(output_folder, str(timelines[j].year)+'.'+ty_list.iloc[idx,0]+'.{:s}.pkl'.format(dt.datetime.strftime(timelines[j]+dt.timedelta(minutes=10*i), format='%Y%m%d%H%M')))
                Z.to_pickle(output_path, compression=args.compression)
        
        if plot_fig:
            fig.savefig(os.path.join(args.weather_figures_folder, str(timelines[j].year)+'.'+
                                     ty_list.iloc[idx,0]+'.{:s}.png'.format(dt.datetime.strftime(timelines[j],format='%Y%m%d%H%M'))),
                                     dpi=100, bbox_inches='tight')
            fig.clf()

蘇拉
蘇力
潭美
康芮
天兔
菲特


## 求最大最小值

In [ ]:
timestep = ty_list['Time of canceling'] - ty_list['Time of issuing']

column = pd.Index(args.I_x_list, name='lon')
index = pd.Index(args.I_y_list, name='lat')

max_value = np.zeros(6)
min_value = np.ones(6)*10000

for idx, i in enumerate(timestep):
    print(ty_list.iloc[idx,2].year, ty_list.iloc[idx,1])
    timelines = [ty_list['Time of issuing'][idx]+dt.timedelta(hours=x) for x in range(i.days*24+i.seconds//3600+1)]
    data_man = os.path.join(args.weather_raw_data_folder, str(ty_list.iloc[idx,2].year)+'.Data', str(ty_list.iloc[idx,2].year)+'_Station_hr_man', ty_list.iloc[idx,0]+'.txt')
    data_auto = os.path.join(args.weather_raw_data_folder, str(ty_list.iloc[idx,2].year)+'.Data', str(ty_list.iloc[idx,2].year)+'_Station_hr_auto', ty_list.iloc[idx,0]+'.txt')
    data = read_data(data_man, data_auto, sta_list)
    # max
    data['PS01'] = localP_2_seaP(p=data['PS01'].to_numpy(), h=data['height'].to_numpy(), t=data['TX01'].to_numpy())
    
    max_tmp = data.iloc[:,1:-3].max().to_numpy()
    max_value[max_tmp > max_value] = max_tmp[max_tmp > max_value]
    
    # min
    min_tmp = data.iloc[:,1:-3].min().to_numpy()
    min_value[min_tmp < min_value] = min_tmp[min_tmp < min_value]
    
outputpath = os.path.join(args.weather_folder, 'overall.csv')
pd.DataFrame([max_value,min_value], index=pd.Index(['max','min'], name='Measures'), columns=pd.Index(data.columns[1:-3])).to_csv(outputpath)

In [ ]:
pd.DataFrame([max_value,min_value], index=['max','min'], columns=data.columns[1:-3])